In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Dense, InputLayer, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import cv2
import math
import os
from glob import glob
from scipy import stats as s

In [2]:
train = pd.read_csv('train_new.csv')
train = train.sample(n=10000).reset_index(drop=True)
train.head()

,image,class
0,v_BrushingTeeth_g22_c03.avi_frame2.jpg,BrushingTeeth
1,v_Rowing_g20_c04.avi_frame0.jpg,Rowing
2,v_BrushingTeeth_g13_c04.avi_frame1.jpg,BrushingTeeth
3,v_HammerThrow_g09_c05.avi_frame4.jpg,HammerThrow
4,v_Drumming_g23_c07.avi_frame6.jpg,Drumming


In [3]:
# creating an empty list
train_image = []

# for loop to read and store frames
for i in tqdm(range(train.shape[0])):
    # loading the image and keeping the target size as (224,224,3)
    img = image.load_img('train_1/'+train['image'][i], target_size=(224,224,3))
    # converting it to array
    img = image.img_to_array(img)
    # normalizing the pixel value
    img = img/255
    # appending the image to the train_image list
    train_image.append(img)
    
# converting the list to numpy array
X = np.array(train_image)

# shape of the array
X.shape

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:46<00:00, 213.72it/s]


(10000, 224, 224, 3)

In [4]:
# separating the target
y = train['class']

# creating the training and validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify = y)

In [5]:
# creating dummies of target variable for train and validation set
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [6]:
y_train.shape

(8000, 101)

In [7]:
# creating the base model of pre-trained VGG16 model
base_model = InceptionResNetV2(weights='imagenet', include_top=False)

In [8]:
# extracting features for training frames
X_train = base_model.predict(X_train)
X_train.shape

(8000, 5, 5, 1536)

In [9]:
# extracting features for validation frames
X_test = base_model.predict(X_test)
X_test.shape

(2000, 5, 5, 1536)

In [10]:
# reshaping the training as well as validation frames in single dimension
# X_train = X_train.reshape(8000, 5*5*1536)
# X_test = X_test.reshape(2000, 5*5*1536)

In [11]:
# normalizing the pixel values
max = X_train.max()
X_train = X_train/max
X_test = X_test/max

In [12]:
# shape of images
X_train.shape

(8000, 5, 5, 1536)

In [13]:
#defining the model architecture
model = Sequential()
model.add(Conv2D(16,(3,3),padding='same',input_shape=(5,5,1536)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(101, activation='softmax'))

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_203 (Conv2D)          (None, 5, 5, 16)          221200    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 2, 2, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 1024)              66560     
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0

In [15]:
# defining a function to save the weights of best model
from tensorflow.keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('InceptionResNetV2weight_Refined.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [16]:
# compiling the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [17]:
# training the model
model.fit(X_train, y_train, epochs=200, validation_data=(X_test, y_test), callbacks=[mcp_save], batch_size=128)

Epoch 1/200
63/63 [==============================] - 4s 62ms/step - loss: 4.5065 - accuracy: 0.0296 - val_loss: 4.1690 - val_accuracy: 0.0820
Epoch 2/200
63/63 [==============================] - 4s 58ms/step - loss: 3.9416 - accuracy: 0.1147 - val_loss: 3.4615 - val_accuracy: 0.2235
Epoch 3/200
63/63 [==============================] - 4s 58ms/step - loss: 3.4156 - accuracy: 0.2021 - val_loss: 2.8675 - val_accuracy: 0.3210
Epoch 4/200
63/63 [==============================] - 4s 58ms/step - loss: 2.9611 - accuracy: 0.2805 - val_loss: 2.4601 - val_accuracy: 0.3945
Epoch 5/200
63/63 [==============================] - 4s 57ms/step - loss: 2.6273 - accuracy: 0.3390 - val_loss: 2.1662 - val_accuracy: 0.4480
Epoch 6/200
63/63 [==============================] - 4s 57ms/step - loss: 2.3466 - accuracy: 0.3855 - val_loss: 1.9563 - val_accuracy: 0.4895
Epoch 7/200
63/63 [==============================] - 4s 58ms/step - loss: 2.0865 - accuracy: 0.4308 - val_loss: 1.7969 - val_accuracy: 0.5285
Epoch 

63/63 [==============================] - 4s 57ms/step - loss: 0.2229 - accuracy: 0.9276 - val_loss: 1.1724 - val_accuracy: 0.8045
Epoch 59/200
63/63 [==============================] - 4s 60ms/step - loss: 0.2362 - accuracy: 0.9280 - val_loss: 1.3063 - val_accuracy: 0.7855
Epoch 60/200
63/63 [==============================] - 4s 59ms/step - loss: 0.2167 - accuracy: 0.9305 - val_loss: 1.3846 - val_accuracy: 0.7830
Epoch 61/200
63/63 [==============================] - 4s 60ms/step - loss: 0.2561 - accuracy: 0.9235 - val_loss: 1.1553 - val_accuracy: 0.7940
Epoch 62/200
63/63 [==============================] - 4s 61ms/step - loss: 0.2344 - accuracy: 0.9280 - val_loss: 1.4742 - val_accuracy: 0.7575
Epoch 63/200
63/63 [==============================] - 4s 58ms/step - loss: 0.2234 - accuracy: 0.9326 - val_loss: 1.2135 - val_accuracy: 0.7955
Epoch 64/200
63/63 [==============================] - 4s 57ms/step - loss: 0.1960 - accuracy: 0.9379 - val_loss: 1.3347 - val_accuracy: 0.7855
Epoch 65/200

63/63 [==============================] - 4s 63ms/step - loss: 0.1224 - accuracy: 0.9669 - val_loss: 1.5244 - val_accuracy: 0.8025
Epoch 116/200
63/63 [==============================] - 4s 61ms/step - loss: 0.1294 - accuracy: 0.9635 - val_loss: 1.5223 - val_accuracy: 0.7950
Epoch 117/200
63/63 [==============================] - 4s 63ms/step - loss: 0.1491 - accuracy: 0.9584 - val_loss: 1.4338 - val_accuracy: 0.7990
Epoch 118/200
63/63 [==============================] - 4s 64ms/step - loss: 0.1270 - accuracy: 0.9666 - val_loss: 1.5853 - val_accuracy: 0.7980
Epoch 119/200
63/63 [==============================] - 4s 61ms/step - loss: 0.1229 - accuracy: 0.9666 - val_loss: 1.3852 - val_accuracy: 0.8110
Epoch 120/200
63/63 [==============================] - 4s 63ms/step - loss: 0.0948 - accuracy: 0.9745 - val_loss: 1.5903 - val_accuracy: 0.7940
Epoch 121/200
63/63 [==============================] - 4s 62ms/step - loss: 0.1347 - accuracy: 0.9670 - val_loss: 1.7282 - val_accuracy: 0.7970
Epoch 

Epoch 172/200
63/63 [==============================] - 4s 60ms/step - loss: 0.1349 - accuracy: 0.9668 - val_loss: 1.8695 - val_accuracy: 0.7900
Epoch 173/200
63/63 [==============================] - 4s 59ms/step - loss: 0.1105 - accuracy: 0.9706 - val_loss: 1.6737 - val_accuracy: 0.8110
Epoch 174/200
63/63 [==============================] - 4s 60ms/step - loss: 0.0993 - accuracy: 0.9732 - val_loss: 1.5597 - val_accuracy: 0.8045
Epoch 175/200
63/63 [==============================] - 4s 60ms/step - loss: 0.1072 - accuracy: 0.9720 - val_loss: 1.7239 - val_accuracy: 0.7995
Epoch 176/200
63/63 [==============================] - 4s 60ms/step - loss: 0.1357 - accuracy: 0.9681 - val_loss: 1.4885 - val_accuracy: 0.7960
Epoch 177/200
63/63 [==============================] - 4s 60ms/step - loss: 0.0940 - accuracy: 0.9754 - val_loss: 1.5640 - val_accuracy: 0.8060
Epoch 178/200
63/63 [==============================] - 4s 60ms/step - loss: 0.0889 - accuracy: 0.9755 - val_loss: 1.5841 - val_accuracy:

In [18]:
# loading the trained weights
model.load_weights("InceptionResNetV2weight_Refined.hdf5")

In [19]:
# compiling the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [20]:
# getting the test list
f = open("ucfTrainTestlist/testlist02.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating the dataframe
test = pd.DataFrame()
test['video_name'] = videos
test = test[:-1]
test_videos = test['video_name']
test.head()

,video_name
0,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01.avi
1,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c02.avi
2,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c03.avi
3,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c04.avi
4,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c05.avi


In [21]:
# creating the tags
train = pd.read_csv('train_new.csv')
y = train['class']
y = pd.get_dummies(y)

In [22]:
# creating two lists to store predicted and actual tags
predict = []
actual = []

# for loop to extract frames from each test video
for i in tqdm(range(test_videos.shape[0])):
    count = 0
    videoFile = test_videos[i]
    cap = cv2.VideoCapture('UCF-101/'+videoFile.split(' ')[0])   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    x=1
    # removing all other files from the temp folder
    files = glob('temp/*')
    for f in files:
        os.remove(f)
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            # storing the frames of this particular video in temp folder
            filename ='temp/' + "_frame%d.jpg" % count;count+=1
            cv2.imwrite(filename, frame)
    cap.release()
    
    # reading all the frames from temp folder
    images = glob("temp/*.jpg")
    
    prediction_images = []
    for i in range(len(images)):
        img = image.load_img(images[i], target_size=(224,224,3))
        img = image.img_to_array(img)
        img = img/255
        prediction_images.append(img)
        
    # converting all the frames for a test video into numpy array
    prediction_images = np.array(prediction_images)
    # extracting features using pre-trained model
    prediction_images = base_model.predict(prediction_images)
    # converting features in one dimensional array
    prediction_images = prediction_images.reshape(prediction_images.shape[0],5,5,1536)
    # predicting tags for each array
    prediction = model.predict_classes(prediction_images)
    # appending the mode of predictions in predict list to assign the tag to the video
    predict.append(y.columns.values[s.mode(prediction)[0][0]])
    # appending the actual tag of the video
    actual.append(videoFile.split('/')[1].split('_')[1])

  0%|                                                                                         | 0/3734 [00:00<?, ?it/s]

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


100%|██████████████████████████████████████████████████████████████████████████████| 3734/3734 [33:39<00:00,  1.85it/s]


In [23]:
# checking the accuracy of the predicted tags
from sklearn.metrics import accuracy_score
accuracy_score(predict, actual)*100

76.29887520085698

# Model Evaluation

Investigating the different architectures to find out if a deeper neural network in addition to the pre-trained model will improve model performance